<a href="https://colab.research.google.com/github/PierrePssi/Projet_MLOPS/blob/Projet_MLOPS_Thao/ScrapeOscarReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ceci est un code de scraping qui extrait les commentaires Allociné sur les films nominés aux Oscar entre 2020 et 2024.

In [ ]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to upload your file


Saving Film_list2.xlsx to Film_list2 (2).xlsx


In [ ]:
#09022025 14H48
import requests
import lxml.html as lh
import pandas as pd
import csv

# Load the Excel file containing film names and URLs
file_name = "Film_list2 (2).xlsx"  # Change this to the actual file name
df_films = pd.read_excel(file_name)

# XPath for extracting reviews
tags = ['//span[@class="stareval-note"]',  # Rating
        '//div[@class="content-txt review-card-content"]']  # Review text

cols = ['Note', 'Description']
uri_pages = '?page='

# Function to scrape a single page
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    content = []
    for tag in tags:
        content.append(doc.xpath(tag))

    df_list = []
    for i, tag_content in enumerate(content):
        tmp = pd.DataFrame([tag_content[j].text_content().strip() for j in range(len(tag_content))], columns=[cols[i]])
        tmp['key'] = tmp.index
        df_list.append(tmp)

    df = df_list[0]
    for i in range(1, len(df_list)):
        df = df.merge(df_list[i], on='key', how='left')

    df.drop(columns=['key'], inplace=True)
    return df

# Function to scrape multiple pages
def getPages(nbPages, url, film_name):
    all_reviews = pd.DataFrame()

    # Ensure nbPages is an integer
    if pd.notna(nbPages) and isinstance(nbPages, (int, float)):
        nbPages = int(nbPages)
    else:
        return all_reviews  # Return empty DataFrame if nbPages is not valid

    for i in range(1, nbPages + 1):
        page_reviews = getPage(url + uri_pages + str(i))
        page_reviews['Film_Name'] = film_name  # Add film name as a column
        all_reviews = pd.concat([all_reviews, page_reviews], ignore_index=True)

    return all_reviews

# List to store all film reviews
all_films_reviews = []

# Loop through all films in the Excel file
for index, row in df_films.iterrows():
    film_name = row['Film_Name']  # Keep original name
    url = row['URL']
    nbPages = row['Pages']

    print(f"Scraping {film_name} ({nbPages} pages)...")

    reviews = getPages(nbPages, url, film_name)
    all_films_reviews.append(reviews)

# Concatenate all reviews into a single DataFrame
final_reviews_df = pd.concat(all_films_reviews, ignore_index=True)

# Save the final combined CSV file
final_reviews_df.to_csv("all_film_reviews.csv", index=False, sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_NONNUMERIC)

print("✅ Scraping completed and all reviews are saved in 'all_film_reviews.csv'!")

Scraping Oppenheimer (145 pages)...
Scraping Poor Things (49 pages)...
Scraping Killers of the Flower Moon (68 pages)...
Scraping Barbie (131 pages)...
Scraping The Holdovers (18 pages)...
Scraping Maestro (8 pages)...
Scraping Anatomy of a Fall (102 pages)...
Scraping Past Lives (16 pages)...
Scraping American Fiction (2 pages)...
Scraping The Zone of Interest (50 pages)...
Scraping Everything Everywhere All at Once (60 pages)...
Scraping Top Gun: Maverick (136 pages)...
Scraping The Banshees of Inisherin (29 pages)...
Scraping Elvis (58 pages)...
Scraping Avatar: The Way of Water (204 pages)...
Scraping The Fabelmans (60 pages)...
Scraping Tár (24 pages)...
Scraping Black Panther: Wakanda Forever (55 pages)...
Scraping Glass Onion: A Knives Out Mystery (25 pages)...
Scraping Triangle of Sadness (41 pages)...
Scraping CODA (4 pages)...
Scraping The Power of the Dog (14 pages)...
Scraping Belfast (13 pages)...
Scraping West Side Story (29 pages)...
Scraping King Richard (16 pages)...
S